In [1]:
access_token = 'EAAH24PQU4egBADGV6Xd8EnwaCmEjvlKZCLYsymsSZBVCPBvvZBtcOnMuotunZAGh4BBvAP9dHYLAnS5J70UOIXDgEbxhgHbGXOTWGAESBfSlJGj6GDZAlDvh5KYzAztgsaOxbGSlVLaT29P00gmsSLBNGXiSZBT71mRUa5rsFo2I8ZC6d4I0oBFFZCaqbyKbbCMZD'
host = '78.46.226.67'
port = '5432'
database ='postgres'
user='kirill_k'
password='swfr35$#'

address = 'postgresql://' + user +':' + password + '@' + host +':' + port + '/' + database

In [2]:
!pip install facebook-business

import psycopg2

from tqdm import tqdm

import pandas as pd
import numpy as np

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.exceptions import FacebookRequestError, FacebookBadObjectError
from facebook_business.adobjects.user import User
from facebook_business.api import FacebookAdsApi
from psycopg2 import OperationalError

import json

import datetime
from datetime import timedelta, datetime, date, time
import time

     |████████████████████████████████| 1.0MB 4.5MB/s 
     |████████████████████████████████| 10.1MB 14.8MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=10a4e488ce45da0ab0022600205fc161baf46fee4bbe05133642a21ec50bdaf7
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for curlify: filename=curlify-2.2.1-cp36-none-any.whl size=2660 sha256=8ff19d3ec98f4725f3a547576ac5b98558cc7547303474644cac591bb5872f19
  Stored in directory: /root/.cache/pip/wheels/b7/5e/e5/6f90a2b21faf7b71748fcba58920ad77850c25b8f70927d418
Successfully built pycountry curlify


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
def connection_to_database(host, port, database, user, password):
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    cur = connection.cursor()
    return cur, connection

def reconnection_to_database(host, port, database, user, password, connection):
    if connection.closed > 0:
        connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        cur = connection.cursor()
    else:
        try:
            connection.cursor().execute('SELECT 1')
        except OperationalError:
            connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
            cur = connection.cursor()
    return connection.cursor(), connection

def store_object(object_data, table):
    fields = ','.join(object_data.keys())
    values = ','.join(map(lambda x:f"%({x})s",object_data.keys()))
    query =   f"INSERT INTO {table} ({fields}) values ({values})"
#    cur.execute(query,object_data)
    connection.commit()
    
def delete_object(object_id, table):
    query_to_delete = f"""delete  from {table} where id = %s"""  
    cur.execute(query_to_delete, vars=(object_id, ))
    
def store_deleted_object(object_id, object_type, account_id, table):
    today = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    query =  f"INSERT INTO {table} values (%s, %s, %s, %s)"
    cur.execute(query, vars=(object_id, account_id, object_type,today,))
    connection.commit()

def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False, sort_keys=True)

def modify_object(object_data):
    for key in object_data.keys():
        if type(object_data[key]) in (list, dict):
            object_data[key] = dict2json(object_data[key])
    object_data['recording_date'] = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    return object_data

In [4]:
FacebookAdsApi.init(access_token=access_token)
me = User(fbid='me')
accounts = list(me.get_ad_accounts())
accounts = list(map(lambda x:x.export_all_data(), accounts))
account_ids = list(map(lambda x:x['account_id'], accounts))

In [5]:
print('start processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

FacebookAdsApi.init(access_token=access_token)

try:
    cur.close()
    cur, connection = connection_to_database(host, port, database, user, password)
except NameError:
    cur, connection = connection_to_database(host, port, database, user, password)

me = User(fbid='me')
accounts = list(me.get_ad_accounts())
accounts = list(map(lambda x:x.export_all_data(), accounts))
account_ids = list(map(lambda x:x['account_id'], accounts))

object_type = 'account'
table = 'property_' + object_type + 's'

cur.execute(f""" CREATE TABLE if not exists {table}(
        id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     name VARCHAR(200), business_street VARCHAR(100),
        tax_id_status INT, attribution_spec JSONB, is_in_3ds_authorization_enabled_market BOOLEAN,
        end_advertiser_name VARCHAR(50), end_advertiser VARCHAR(50), agency_client_declaration JSONB,
        failed_delivery_checks JSONB,
        is_tax_id_required BOOLEAN, min_campaign_group_spend_cap VARCHAR(20), timezone_name VARCHAR(50),
        user_role VARCHAR(50), business JSONB, age FLOAT, capabilities JSONB, business_name VARCHAR(200), 
        created_time timestamp,   is_attribution_spec_system_default BOOLEAN, timezone_id INT,
        timezone_offset_hours_utc INT, tos_accepted JSONB, is_personal INT, fb_entity INT, 
        rf_spec JSONB, can_create_brand_lift_study BOOLEAN, business_city VARCHAR(20),
        offsite_pixels_tos_accepted BOOLEAN, spend_cap VARCHAR(20), business_country_code VARCHAR(2),
        is_notifications_enabled BOOLEAN, amount_spent VARCHAR(20), business_zip VARCHAR(20), 
        min_daily_budget INT, account_status INT, is_direct_deals_enabled BOOLEAN, owner VARCHAR(50), 
        disable_reason INT, currency VARCHAR(5), is_in_middle_of_local_entity_migration BOOLEAN, 
        business_street2 VARCHAR(50), has_migrated_permissions BOOLEAN, business_state  VARCHAR(20),
        is_prepay_account BOOLEAN, balance  VARCHAR(20), tax_id_type VARCHAR(20),
        recording_date timestamp without time zone)""")
connection.commit()

cur.execute(f""" CREATE TABLE if not exists deleted_objects(
    object_id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     
    object_type VARCHAR(20),    recording_date timestamp without time zone)""")
connection.commit()

fields = list(AdAccount.Field.__dict__.keys())
exclude_list = ['__module__', 'ad_account_creation_request', 'ad_account_promotable_objects',
'funding_source', 'funding_source_details', 'has_page_authorized_adaccount', 'show_checkout_experience', '__doc__']

fields = list(set(fields)-set(exclude_list ))

query = """SELECT DISTINCT t.*
FROM (SELECT DISTINCT actions.account_id as account_id, 
property_accounts.recording_date is NULL as has_not_data
FROM actions LEFT JOIN property_accounts on actions.account_id = property_accounts.account_id
WHERE object_type = 'ACCOUNT' and 
(property_accounts.recording_date is NULL or actions.event_time > property_accounts.recording_date)
UNION
SELECT DISTINCT insights.account_id, False as has_not_data
from insights
left join property_accounts on insights.account_id = property_accounts.account_id
where property_accounts.account_id is null) t
LEFT JOIN deleted_objects on  t.account_id = deleted_objects.object_id
WHERE deleted_objects.object_id IS NULL"""

account_list = pd.read_sql(query, connection)
data = pd.read_sql(query, connection)
data = data[data['account_id'].isin(account_ids)]
accounts = dict(zip(list(data['account_id']), data['has_not_data']))
accounts =  dict(zip(list(data['account_id']), list(data['account_id'])))

print('start uploading \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

for account_id in tqdm(accounts.keys()):
    try:
        account_data = AdAccount('act_' + account_id).api_get(fields = fields).export_all_data()
        account_data = modify_object(account_data)
        cur, connection = reconnection_to_database(host, port, database, user, password, connection)
        if accounts[account_id]:
            store_object(account_data, table)
        else:
            delete_object('act_' + account_id, table)
            store_object(account_data, table)
        connection.commit()
        
    except FacebookRequestError as error:
        pass
#         code = error._body['error']['code']
#         subcode = error._body['error']['error_subcode']
#         if code == 100 and (subcode == 33 or subcode == 1487221):
#             cur, connection = reconnection_to_database(host, port, database, user, password, connection)
#             store_deleted_object(adset_id, object_type, accounts[adset_id], 'deleted_objects')
#        print('An unknown error occurred for adset ' + adset_id)

print('start refreshing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

# cur, connection = reconnection_to_database(host, port, database, user, password, connection)
# query = """REFRESH MATERIALIZED VIEW mview_adset_atributes"""
# cur.execute(query)
# connection.commit()

print('end processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

start processing 	2020-07-13 19:39:27


  0%|          | 0/11 [00:00<?, ?it/s]

start uploading 	2020-07-13 19:39:29


100%|██████████| 11/11 [00:07<00:00,  1.56it/s]

start refreshing 	2020-07-13 19:39:36
end processing 	2020-07-13 19:39:36
